# 음성 데이터 전처리

In [ ]:
# !pip install noisereduce

In [ ]:
# import os
# import librosa
# import soundfile as sf
# import noisereduce as nr

# def preprocess_audio(file_path, output_path):
#     # 1. 샘플링 레이트 고정 및 오디오 로드
#     sr = 16000
#     y, sr = librosa.load(file_path, sr=sr, mono=True)

#     # 2. 노이즈 제거 (필요시 파라미터 조정)
#     y_denoised = nr.reduce_noise(y=y, sr=sr)

#     # 3. 음량 정규화 및 클리핑 방지
#     rms_level = librosa.feature.rms(y=y_denoised)[0]
#     rms_mean = rms_level.mean()
    
#     if rms_mean < 0.05:  # 음량이 너무 작은 경우 증폭
#         gain_factor = 0.05 / rms_mean
#         y_denoised = y_denoised * gain_factor
    
#     # 4. 무음 구간 제거 (노이즈 제거 후)
#     y_trimmed, _ = librosa.effects.trim(y_denoised, top_db=20)

#     # 5. 전처리된 파일 저장 (float32로 저장)
#     output_file = os.path.join(output_path, os.path.basename(file_path))
#     sf.write(output_file, y_trimmed, sr, subtype='FLOAT')
#     print(f"Processed file saved at: {output_file}")

# # 폴더 내 모든 .wav 파일 전처리
# def preprocess_directory(input_dir, output_dir):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     for file_name in os.listdir(input_dir):
#         if file_name.endswith(".wav"):
#             file_path = os.path.join(input_dir, file_name)
#             preprocess_audio(file_path, output_dir)

# # 사용 예시
# input_directory = "/kaggle/input/total-data/Total_train"
# output_directory = "/kaggle/working/Processed_audio_1114_train"
# preprocess_directory(input_directory, output_directory)


In [ ]:
# import os
# import librosa
# import soundfile as sf
# import noisereduce as nr

# def preprocess_audio(file_path, output_path):
#     # 1. 샘플링 레이트 고정 및 오디오 로드
#     sr = 16000
#     y, sr = librosa.load(file_path, sr=sr, mono=True)

#     # 2. 노이즈 제거 (필요시 파라미터 조정)
#     y_denoised = nr.reduce_noise(y=y, sr=sr)

#     # 3. 음량 정규화 및 클리핑 방지
#     rms_level = librosa.feature.rms(y=y_denoised)[0]
#     rms_mean = rms_level.mean()
    
#     if rms_mean < 0.05:  # 음량이 너무 작은 경우 증폭
#         gain_factor = 0.05 / rms_mean
#         y_denoised = y_denoised * gain_factor
    
#     # 4. 무음 구간 제거 (노이즈 제거 후)
#     y_trimmed, _ = librosa.effects.trim(y_denoised, top_db=20)

#     # 5. 전처리된 파일 저장 (float32로 저장)
#     output_file = os.path.join(output_path, os.path.basename(file_path))
#     sf.write(output_file, y_trimmed, sr, subtype='FLOAT')
#     print(f"Processed file saved at: {output_file}")

# # 폴더 내 모든 .wav 파일 전처리
# def preprocess_directory(input_dir, output_dir):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     for file_name in os.listdir(input_dir):
#         if file_name.endswith(".wav"):
#             file_path = os.path.join(input_dir, file_name)
#             preprocess_audio(file_path, output_dir)

# # 사용 예시
# input_directory = "/kaggle/input/total-data/Total_test"
# output_directory = "/kaggle/working/Processed_audio_1114_test"
# preprocess_directory(input_directory, output_directory)


In [ ]:
#!ls ./checkpoints

# 음성 감정 분석

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import math
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from transformers.optimization import AdamW, get_constant_schedule_with_warmup
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, StochasticWeightAveraging, LearningRateMonitor
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer, AutoFeatureExtractor, HubertForSequenceClassification, AutoConfig

In [ ]:
# 1. 정확도 계산
def accuracy(preds, labels):
    return (preds == labels).float().mean()

# 2. 오디오 파일 로드
def getAudios(df,num):
    audios = []
    dir = '/kaggle/input/total-data/Total_train/' if num == 1 else '/kaggle/input/total-data/Total_test/'
    for idx, row in tqdm(df.iterrows(),total=len(df)):
        try:
            # 16K 샘플링 속도로 오디오 파일 로드
            audio, _ = librosa.load(dir + row['wav_id'] + '.wav', sr=SAMPLING_RATE)
            audios.append(audio)
        except Exception as e:
            # 오류 발생 시 해당 파일을 건너뛰고 계속 진행
            print(f"Error loading {row['wav_id']}.wav: {e}")
            continue
    return audios

# 3. 배치 데이터 효율적 처리 - 패딩 관련 수정?
def collate_fn(samples):
    batch_labels = [sample['label'] for sample in samples]
    batch_audio_values = [torch.tensor(sample['audio_values']) for sample in samples]
    batch_audio_attn_masks = [torch.tensor(sample['audio_attn_mask']) for sample in samples]

    batch = {
        'label': torch.tensor(batch_labels),
        'audio_values': pad_sequence(batch_audio_values, batch_first=True),
        'audio_attn_mask': pad_sequence(batch_audio_attn_masks, batch_first=True),
    }

    return batch
    #batch_labels = torch.tensor(batch_labels)
    # 가장 긴 시퀀스를 기준으로 길이 맞춤, 빈 공간은 0으로 채움 (padding)
    #batch_audio_values = pad_sequence(batch_audio_values, batch_first=True)
    #batch_audio_attn_masks = pad_sequence(batch_audio_attn_masks, batch_first=True)

    #batch = {
    #    'label': batch_labels,
    #    'audio_values': batch_audio_values,
    #    'audio_attn_mask': batch_audio_attn_masks,
    #}

    #return batch

In [ ]:
!pip install audiomentations

In [ ]:
# 4. 데이터셋 클래스
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, model_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2) * -(math.log(10000.0) / model_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        """
        x: Tensor of shape (batch_size, seq_len, model_dim)
        """
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len]


class MyDataset(Dataset):
    def __init__(self, audio, audio_feature_extractor, label=None, augment=True):
        self.label = np.array(label if label is not None else [0] * len(audio)).astype(np.int64)
        self.audio = audio
        # 오디오 데이터를 모델이 이해할 수 있는 형태로 변환
        self.audio_feature_extractor = audio_feature_extractor
        self.augment = augment
        
        # 데이터 증강 파이프라인
        self.augmenter = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
            PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
            Shift(min_shift=-0.5, max_shift=0.5, p=0.5),  # 변경된 인자
        ])

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        label = self.label[idx]
        audio = self.audio[idx]

        if self.augment:
            audio = self.augmenter(samples=audio, sample_rate=SAMPLING_RATE)
        audio_feature = self.audio_feature_extractor(raw_speech=audio, return_tensors='np', sampling_rate=SAMPLING_RATE)
        audio_values, audio_attn_mask = audio_feature['input_values'][0], audio_feature['attention_mask'][0]

        item = {
            'label':label,
            'audio_values':audio_values,
            'audio_attn_mask':audio_attn_mask,
        }
    
        return item
        
from sklearn.metrics import accuracy_score

# 5. 음성 분류 모델 클래스
class MyLitModel(pl.LightningModule):
    def __init__(self, audio_model_name, num_labels, transformer_dim=512, nhead=4, num_layers=1, n_layers=1, projector=True, classifier=True, dropout=0.2, lr_decay=1, fold_idx=0):
        super(MyLitModel, self).__init__()
        self.audio_model = AutoModel.from_pretrained(audio_model_name)
        # Positional Encoding 추가
        self.positional_encoding = PositionalEncoding(self.audio_model.config.hidden_size)
        # Transformer 추가
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=self.audio_model.config.hidden_size,
                nhead=nhead,
                dim_feedforward=transformer_dim,
                dropout=dropout,
                activation='gelu' #relu
            ),
            num_layers=num_layers
        )
        
        # 분류기 추가
        self.classifier = nn.Linear(self.audio_model.config.hidden_size, num_labels)
        self.dropout = nn.Dropout(dropout)
        
        self.fold_idx = fold_idx
        self.lr_decay = lr_decay
        
        # 손실 및 정확도 저장을 위한 리스트
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        
    def forward(self, audio_values, audio_attn_mask):
        # Hubert에서 특성 추출
        output = self.audio_model(input_values=audio_values, attention_mask=audio_attn_mask)
        hidden_states = output.last_hidden_state  # (batch_size, seq_len, hidden_size)
        # Transformer에 입력
        hidden_states = self.positional_encoding(hidden_states) ##
        hidden_states = hidden_states.permute(1, 0, 2)  # Transformer 입력 형식: (seq_len, batch_size, hidden_size)
        transformer_out = self.transformer(hidden_states)  # (seq_len, batch_size, hidden_size)
        transformer_out = transformer_out.permute(1, 0, 2)  # 다시 원래 형식으로 변경: (batch_size, seq_len, hidden_size)

        # 마지막 타임스텝의 출력만 사용하거나 평균 풀링
        pooled_out = transformer_out.mean(dim=1)  # (batch_size, hidden_size)
        
        # 분류기 통과
        logits = self.classifier(self.dropout(pooled_out))  # (batch_size, num_labels)
        return logits

    def training_step(self, batch, batch_idx):
        audio_values = batch['audio_values']
        audio_attn_mask = batch['audio_attn_mask']
        labels = batch['label']

        logits = self(audio_values, audio_attn_mask)
        # 이진 교차 엔트로피 손실 계산
        loss = nn.BCEWithLogitsLoss()(logits.view(-1), labels.float())
        
        preds = torch.round(torch.sigmoid(logits.view(-1)))
        # train_acc 게산
        acc = accuracy(preds, labels.float())

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        # 손실 및 정확도 저장
        self.train_losses.append(loss.item())
        self.train_accuracies.append(acc.item())
        
        return loss

    def validation_step(self, batch, batch_idx):
        audio_values = batch['audio_values']
        audio_attn_mask = batch['audio_attn_mask']
        labels = batch['label']

        logits = self(audio_values, audio_attn_mask)
        # 검증 데이터 손실 계산
        loss = nn.BCEWithLogitsLoss()(logits.view(-1), labels.float())

        preds = torch.round(torch.sigmoid(logits.view(-1)))
        # 검증 데이터 정확도 계산
        acc = accuracy(preds, labels.float())

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        # 손실 및 정확도 저장
        self.val_losses.append(loss.item())
        self.val_accuracies.append(acc.item())
        
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        audio_values = batch['audio_values']
        audio_attn_mask = batch['audio_attn_mask']
        # 최종 예측값 
        logits = self(audio_values, audio_attn_mask)
        preds = torch.round(torch.sigmoid(logits.view(-1)))

        return preds

    def configure_optimizers(self):
        lr = 1e-5
        layer_decay = self.lr_decay
        weight_decay = 0.01
        # 계층별 학습률 감쇠 적용해 파라미터 그룹 생성
        llrd_params = self._get_llrd_params(lr=lr, layer_decay=layer_decay, weight_decay=weight_decay)
        optimizer = AdamW(llrd_params)
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True),
            'monitor': 'val_loss',  # 검증 손실 기준으로 감소
            'interval': 'epoch',
            'frequency': 1
        }
        return [optimizer], [scheduler]

    # 학습 종료 후 손실 및 정확도 그래프 저장
    def on_train_epoch_end(self):
        avg_train_loss = self.trainer.callback_metrics['train_loss'].item()
        avg_train_acc = self.trainer.callback_metrics['train_acc'].item()
        self.train_losses.append(avg_train_loss)
        self.train_accuracies.append(avg_train_acc)

    def on_validation_epoch_end(self):
        avg_val_loss = self.trainer.callback_metrics['val_loss'].item()
        avg_val_acc = self.trainer.callback_metrics['val_acc'].item()
        self.val_losses.append(avg_val_loss)
        self.val_accuracies.append(avg_val_acc)
        
    def on_train_end(self):
        # 각 리스트에 맞는 epochs 생성
        train_epochs = range(1, len(self.train_losses) + 1)
        val_epochs = range(1, len(self.val_losses) + 1)
        
        # 훈련 손실 그래프
        plt.figure(figsize=(10, 4))
        plt.plot(train_epochs, self.train_losses, label='Train Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training Loss')
        plt.savefig(f'./fold_{self.fold_idx}_train_loss.png')
        plt.show()
    
        # 검증 손실 그래프
        plt.figure(figsize=(10, 4))
        plt.plot(val_epochs, self.val_losses, label='Validation Loss', color='orange')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Validation Loss')
        plt.savefig(f'./fold_{self.fold_idx}_val_loss.png')
        plt.show()
    
        # 훈련 정확도 그래프
        plt.figure(figsize=(10, 4))
        plt.plot(train_epochs, self.train_accuracies, label='Train Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Training Accuracy')
        plt.savefig(f'./fold_{self.fold_idx}_train_accuracy.png')
        plt.show()
    
        # 검증 정확도 그래프
        plt.figure(figsize=(10, 4))
        plt.plot(val_epochs, self.val_accuracies, label='Validation Accuracy', color='orange')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Validation Accuracy')
        plt.savefig(f'./fold_{self.fold_idx}_val_accuracy.png')
        plt.show()

        
    # 계층별 학습률 감쇠 적용
    def _get_llrd_params(self, lr, layer_decay, weight_decay):
        n_layers = self.audio_model.config.num_hidden_layers
        llrd_params = []
        for name, value in list(self.named_parameters()):
            if ('bias' in name) or ('layer_norm' in name):
                llrd_params.append({"params": value, "lr": lr, "weight_decay": 0.0})
            elif ('emb' in name) or ('feature' in name) : 
                llrd_params.append({"params": value, "lr": lr * (layer_decay**(n_layers+1)), "weight_decay": weight_decay})
            elif 'encoder.layer' in name:
                for n_layer in range(n_layers):
                    if f'encoder.layer.{n_layer}' in name:
                        llrd_params.append({"params": value, "lr": lr * (layer_decay**(n_layer+1)), "weight_decay": weight_decay})
            else:
                llrd_params.append({"params": value, "lr": lr , "weight_decay": weight_decay})
        return llrd_params
    
    # 레이어 재초기화 
    def _do_reinit(self, n_layers=0, projector=True, classifier=True):
        if projector:
            self.audio_model.projector.apply(self._init_weight_and_bias)
        if classifier:
            self.audio_model.classifier.apply(self._init_weight_and_bias)
        
        for n in range(n_layers):
            self.audio_model.hubert.encoder.layers[-(n+1)].apply(self._init_weight_and_bias)
    
    # 레이어 가중치 초기화
    def _init_weight_and_bias(self, module):                        
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.audio_model.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)   

In [ ]:
DATA_DIR = '/kaggle/input/total-csv' # csv 파일 있는 경로
PREPROC_DIR = '/kaggle/working/preproc'# 전처리된 데이터 저장 경로
SUBMISSION_DIR = '/kaggle/working/submission' # 제출 파일 경로
MODEL_DIR = '/kaggle/working/model' # 모델 저장 경로
SAMPLING_RATE = 16000
SEED=0 # 랜덤 시드 값
N_FOLD=10 # 교차 검증
BATCH_SIZE=8
NUM_LABELS = 1 # 이진 분류

In [ ]:
seed_everything(SEED) # 결과 재현성 확보
audio_model_name = 'team-lucid/hubert-base-korean'

# 특성 추출기
audio_feature_extractor = AutoFeatureExtractor.from_pretrained(audio_model_name)
# 마스크 반환하도록 설정해 패딩된 부분 무시
audio_feature_extractor.return_attention_mask=True

In [ ]:
train_df = pd.read_csv(f'{DATA_DIR}/Total_balanced_train_data.csv', encoding='CP949')
test_df = pd.read_csv(f'{DATA_DIR}/Total_balanced_test_data.csv', encoding='CP949')
train_audios = getAudios(train_df,1)
test_audios = getAudios(test_df,0)
train_label = train_df['상황'].apply(lambda x: 1 if x == 'fear' else 0).values

In [ ]:
skf = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)
for fold_idx, (train_indices, val_indices) in enumerate(skf.split(train_label, train_label)):
    # 데이터 분리
    print(train_indices)
    print(len(train_audios))
    train_fold_audios = [train_audios[train_index] for train_index in train_indices]
    val_fold_audios = [train_audios[val_index] for val_index in val_indices]
    
    # 라벨 분리
    train_fold_label = train_label[train_indices]
    val_fold_label = train_label[val_indices]
    
    # 훈련 및 검증 데이터셋 생성
    train_fold_ds = MyDataset(train_fold_audios, audio_feature_extractor, train_fold_label)
    val_fold_ds = MyDataset(val_fold_audios, audio_feature_extractor, val_fold_label)
    train_fold_dl = DataLoader(train_fold_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    val_fold_dl = DataLoader(val_fold_ds, batch_size=BATCH_SIZE * 2, collate_fn=collate_fn)
    
    # 체크포인트 설정
    checkpoint_acc_callback = ModelCheckpoint(
        monitor='val_acc', # 검증 정확도 높은 모델 저장
        dirpath=MODEL_DIR,
        filename=f'fold_{fold_idx}_{{epoch:02d}}-{{val_acc:.4f}}-{{train_acc:.4f}}',
        save_top_k=1,
        mode='max'
    )

    # 모델 초기화
    my_lit_model = MyLitModel(
        audio_model_name=audio_model_name,
        num_labels=1,               # 이진 분류를 위한 num_labels = 1
        n_layers=1, projector=True, classifier=True, dropout=0.2, lr_decay=0.8, fold_idx=fold_idx
    )

    
    # 학습률 모니터링 콜백
    lr_monitor = LearningRateMonitor(logging_interval='epoch')
    
    # 학습 실행 설정
    trainer = pl.Trainer(
        accelerator='cuda', 
        max_epochs=15,
        precision=16, # 16-bit 연산
        val_check_interval=0.50, # 에포크의 20%마다 검증 수행
        callbacks=[checkpoint_acc_callback, lr_monitor], # 최고 성능 모델 저장
    )

    # 학습 실행
    trainer.fit(my_lit_model, train_fold_dl, val_fold_dl)
    


    # 메모리 해제
    del my_lit_model

In [ ]:
from sklearn.metrics import accuracy_score

#테스트 라벨 
test_label = test_df['상황'].apply(lambda x: 1 if x == 'fear' else 0).values
#test_label = test_df['label'].apply(lambda x: 1 if x == 1 else 0).values # 데이콘 전용 
# 테스트 데이터셋 생성
test_ds = MyDataset(test_audios, audio_feature_extractor)
# 데이터셋 로드해 배치 단위로 처리
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE*2, collate_fn=collate_fn)
# 사전 학습된 모델 로드 및 예측 설정
#MODEL_DIR = '/kaggle/input/1117model'
pretrained_models = list(map(lambda x: os.path.join(MODEL_DIR,x),os.listdir(MODEL_DIR)))
print(pretrained_models)

In [ ]:
test_preds = []
trainer = pl.Trainer(
    accelerator='cuda', 
    precision=16,
)

# 각 모델에 대해 예측 수행
for pretrained_model_path in pretrained_models:
    pretrained_model = MyLitModel.load_from_checkpoint(
        pretrained_model_path,
        audio_model_name=audio_model_name,
        num_labels=NUM_LABELS,
    )
    test_pred = trainer.predict(pretrained_model, test_dl)
    test_pred = torch.cat(test_pred).detach().cpu().numpy()
    test_preds.append(test_pred)
    del pretrained_model

    # 모델의 예측값 평균 (모델 앙상블) 혹은 마지막 모델의 예측 사용
    final_pred = np.mean(test_preds, axis=0)  # 예측값의 평균
    final_pred = (final_pred > 0.5).astype(int)  # 0.5 기준으로 이진화
    
    accuracy = accuracy_score(test_label, final_pred)
    print(pretrained_model_path)
    print(f"테스트 데이터 정확도: {accuracy * 100:.2f}%")